<a href="https://colab.research.google.com/github/Lanwei02/Bitcoin-MarketPrice-Prediction/blob/master/sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


pd.set_option('max_colwidth', 500)

In [3]:
!pip install transformers bert torch

     |████████████████████████████████| 1.3MB 8.5MB/s 
     |████████████████████████████████| 2.9MB 21.6MB/s 
     |████████████████████████████████| 890kB 25.4MB/s 
     |████████████████████████████████| 1.1MB 44.9MB/s 
  Created wheel for bert: filename=bert-2.2.0-cp36-none-any.whl size=3756 sha256=e2245378f5cdc6e53633082be98d2958e3529c94c63db9dd91065d1289dc618c
  Stored in directory: /root/.cache/pip/wheels/fe/71/b7/941459453bd38e5d97a8c886361dee19325e9933c9cf88ad46
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=e110bf8ce1c10e149290299a96cb4abc39ca8298588cef3644e7c8b46dafad19
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
  Created wheel for erlastic: filename=erlastic-2.0.0-cp36-none-any.whl size=6789 sha256=276b51317dd7faa6b90024ebac7cb1ec50bb6f0800d40f205e509aa5dfe2f100
  Stored in directory: /root/.cache/pip/wheels/02/62/46/93c713a5f061aeeb4f16eb6bf5ee798816e6ddda70faa7

In [4]:
# Setup root directory
GET_DATA_FROM_COLAB=True
if GET_DATA_FROM_COLAB:
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)
  root_path = '/content/gdrive/My Drive/Geekon2020' 
else: # Local machine
    root_path ='' # e.g., root_path = '/Users/shxu/Documents/hack2020/'
os.chdir(root_path)
!pwd

Mounted at /content/gdrive
/content/gdrive/My Drive/Geekon2020


In [5]:
senti_data = pd.read_csv('sentiment_hbw.csv',
                         names = ['deal_uuid', 'permalink', 'grt_l1_cat_name', 'grt_l2_cat_name', 'grt_l3_cat_name', 'grt_l4_cat_name',
                     'pds_cat_name', 'review', 'dwh_updated_at', 'int_value', 'sentiment_type', 'sentiment_score','user_id', 'client_platform'],
                         delimiter='\t', index_col=False,
                         na_values = ['NA', 'N/A', '\\N'])

In [6]:
senti_data.head(2)

,deal_uuid,permalink,grt_l1_cat_name,grt_l2_cat_name,grt_l3_cat_name,grt_l4_cat_name,pds_cat_name,review,dwh_updated_at,int_value,sentiment_type,sentiment_score,user_id,client_platform
0,77080685-4f91-4af2-ada2-e49703fcf6c4,pure-bliss-beauty,L1 - Local,L2 - Health / Beauty / Wellness,L3 - Maintenance,L4 - Eyelash & Eyebrow Extensions,Eyelash Extensions,Crystal did an amazing job on my lashes. I love them!,2020-02-20 00:24:52.000,5.0,positive,0.70445,12c412d0-9244-11e6-8ccc-002590980646,ios
1,77080685-4f91-4af2-ada2-e49703fcf6c4,pure-bliss-beauty,L1 - Local,L2 - Health / Beauty / Wellness,L3 - Maintenance,L4 - Eyelash & Eyebrow Extensions,Eyelash Extensions,This was a gift for my girlfriend and she absolutely loved it. Crystal did an amazing job on her eye lash extensions. Highly recommend this as she was very personable and professional.,2020-01-31 00:27:04.000,5.0,positive,0.81275,bc2f5190-a4e2-11e5-a3f9-002590980808,ios


## *PreProcessing*

In [7]:
def preprocessing(data):

  # Rename
  data.rename(columns = {'int_value': 'star_rating', 'dwh_updated_at': 'update_timestamp'}, inplace = True)
  
  # Drop NULL Label
  data = data[~data['star_rating'].isna()]

  # Drop 0 star rating deals
  data = data[data['star_rating']>0]

  # Correct data type
  data['update_timestamp'] = pd.to_datetime(data['update_timestamp'])
  data['star_rating'] = data['star_rating'].astype(object)

  # Impute NULL review with empty string
  data['review'].fillna('', inplace = True)



  return data


senti_data_c = senti_data.copy()
senti_data_c = preprocessing(senti_data_c)


In [8]:
senti_data_c[senti_data_c['sentiment_type'].isnull()]['star_rating'].value_counts()

5.0    972
4.0     94
3.0     30
1.0     26
2.0     14
Name: star_rating, dtype: int64

In [9]:
senti_data_c[senti_data_c['sentiment_type'].isnull()]['review'].unique()

array([' ', '\\n', '\\n\\n',
       'My partner stated that the lady had a rough hand and also forced her to pay $10 for each treatment.',
       '\\n\\n\\n', '  ', ' \\n', '     ',
       'Ligia is excellent and very nice!\\nI will definitely visit her again.',
       '\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n\\n',
       'She is professional.  Excellent job to my hair cut and color.  Love it.',
       'Martha is amazing! The girls are so sweet and accommodating. The prices are so good here.',
       '\\n\\n\\n\\n\\n\\n\\n',
       'Elise is amazing!  Truly the best massage I have had in over 15 years!',
       'Had an amazing experience here. No wait and Yvonne took her time and made sure I was satisfied. They also were very clean and sanitized everything along with following all COVID protocol. Would recommend this place to all my friends.',
       '   \\n', '\\n\\n\\n\\n\\n\\n\\n\\n\\n', ' \\n\\n',

In [10]:
# senti_data_c['sentiment_type'].unique()[3]

In [11]:
acc_df = pd.DataFrame(senti_data_c[['star_rating', 'sentiment_type']].value_counts()).rename(columns={0: 'count'}).reset_index()
acc_df['pct'] = acc_df['count']/acc_df['count'].sum()
acc_df['star_rating_sentiment'] = acc_df['star_rating'].map(lambda x: 'positive' if x >=4 else ('negative' if x <= 2 else 'neutral'))

In [12]:
acc_df.sort_values(by=['star_rating_sentiment', 'sentiment_type'])

,star_rating,sentiment_type,count,pct,star_rating_sentiment
7,1.0,negative,5815,0.011931,negative
10,2.0,negative,1627,0.003338,negative
9,1.0,positive,1806,0.003706,negative
13,2.0,positive,1031,0.002115,negative
2,1.0,unknown,19847,0.040722,negative
6,2.0,unknown,7353,0.015087,negative
12,3.0,negative,1302,0.002671,neutral
15,3.0,neutral,1,0.000002,neutral
8,3.0,positive,2615,0.005366,neutral
5,3.0,unknown,8974,0.018413,neutral


In [13]:
acc_df[acc_df['sentiment_type'] == acc_df['star_rating_sentiment']]['pct'].sum()

0.722491808122321

In [14]:
# acc_df.to_csv('prevous_accuracy.csv')

### Train Test Split

### Fine-Tuning BERT for Sentiment Analysis

In [40]:
from sklearn.model_selection import train_test_split

X = senti_data_c['review'].values
y = senti_data_c['star_rating'].values

# 70% train, 20% validation, 10% test
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3, random_state=2020, stratify = y)

In [41]:
X = X_val
y = y_val

X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.333, random_state=2020, stratify = y)

In [110]:
df_train = pd.DataFrame({'review': X_train, 'star_rating': y_train}, columns=['review', 'star_rating'])
df_val = pd.DataFrame({'review': X_val, 'star_rating': y_val}, columns=['review', 'star_rating'])
df_test = pd.DataFrame({'review': X_test, 'star_rating': y_test}, columns=['review', 'star_rating'])

In [111]:
from torch.utils.data import Dataset
from transformers import BertTokenizer
import torch

In [112]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [113]:
# Use BERT tokenizer since it needs to be able to match the tokens to the pre trained words.
# tokens = tokenizer.tokenize(X_train[0])

In [114]:
# print(tokens)
# print(len(tokens))

In [115]:
# BERT inputs typically start with a '[CLS]' tag and end with a '[SEP]' tag. For
# tokens = ['[CLS]'] + tokens + ['[SEP]']

In [116]:
# maxlen = 400

In [117]:
# if len(tokens) < maxlen:
#   # Add the ['PAD'] token
#   tokens = tokens + ['[PAD]' for item in range(maxlen-len(tokens))]
# else:
#   # Truncate the tokens at maxLen - 1 and add a '[SEP]' tag.
#   tokens = tokens[:self.maxlen-1] + ['[SEP]']  

In [118]:
# BERT tokenizer converts the string tokens to their respective IDs.
# token_ids = tokenizer.convert_tokens_to_ids(tokens)

In [119]:
# Converting to pytorch tensors.
# tokens_ids_tensor = torch.tensor(token_ids)

In [120]:
# Masks place a 1 if token != PAD else a 0.
# attn_mask = (tokens_ids_tensor != 0).long()

In [121]:
# attn_mask

## Tokenizer

In [122]:
config = {
    "splitRatio" : 0.8,
    # "maxLength" : 100,
    "maxLength" : 400,
    "printEvery" : 5,
    "outputFolder" : "Models",
    "outputFileName" : "AmazonReviewClassifier.dat",
    "threads" : 4,
    "batchSize" : 64,
    "validationFraction" : 0.0005,
    "epochs" : 5,
    "forceCPU" : False
    }

In [134]:
df = pd.concat([df_train, df_val])

In [135]:
num_classes = df['star_rating'].nunique()
print("Number of Target Output Classes:", num_classes)
totalDatasetSize = len(df)

Number of Target Output Classes: 5


In [138]:
# Group by the column Score. This helps you get distribution of the Review Scores.
symbols = df.groupby('star_rating')

In [142]:
scores_dist = []
for i in range(num_classes):
  scores_dist.append(len(symbols.groups[i+1])/totalDatasetSize)

In [143]:
scores_dist

[0.05628078761718057,
 0.02052280387349729,
 0.026451765497855384,
 0.05361537027013504,
 0.8431292727413318]

In [127]:
# Set GPU for Training
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    config["device"] = device
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla T4


In [128]:
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [129]:
from torch.utils.data import Dataset
from transformers import BertTokenizer
import torch

class ReviewsDataset(Dataset):
    def __init__(self, df, maxlen):
        self.df = df
        # self.df_x = df_x
        # self.df_y = df_y
        # A reset reindexes from 1 to len(df), the shuffled df frames are sparse.
        # self.df.reset_index(drop=True, inplace=True)
        self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        self.maxlen = maxlen

    def __len__(self):
        return(len(self.df))

    def __getitem__(self, index):
        review = self.df.loc[index, 'review']
        # review = self.df_x.loc[index]

        # Classes start from 0.
        # label = int(self.df.loc[index, 'Score']) - 1
        # label = int(self.df_y.loc[index]) - 1
        label = int(self.df.loc[index, 'star_rating']) - 1

        # Use BERT tokenizer since it needs to be able to match the tokens to the pre trained words.
        tokens = self.tokenizer.tokenize(review)

        # BERT inputs typically start with a '[CLS]' tag and end with a '[SEP]' tag. For
        tokens = ['[CLS]'] + tokens + ['[SEP]']

        if len(tokens) < self.maxlen:
            # Add the ['PAD'] token
            tokens = tokens + ['[PAD]' for item in range(self.maxlen-len(tokens))]
        else:
            # Truncate the tokens at maxLen - 1 and add a '[SEP]' tag.
            tokens = tokens[:self.maxlen-1] + ['[SEP]']

        # BERT tokenizer converts the string tokens to their respective IDs.
        token_ids = self.tokenizer.convert_tokens_to_ids(tokens)

        # Converting to pytorch tensors.
        tokens_ids_tensor = torch.tensor(token_ids)

        # Masks place a 1 if token != PAD else a 0.
        attn_mask = (tokens_ids_tensor != 0).long()

        return tokens_ids_tensor, attn_mask, label

In [130]:
train_set = ReviewsDataset(df_train, maxlen = config['maxLength'])
val_set = ReviewsDataset(df_val, maxlen = config['maxLength'])
test_set = ReviewsDataset(df_test, maxlen = config['maxLength'])

In [131]:
from torch.utils.data import DataLoader

In [132]:
train_loader = DataLoader(train_set, batch_size = config["batchSize"], num_workers = config["threads"])
val_loader = DataLoader(val_set, batch_size = config["batchSize"], num_workers = config["threads"])
test_loader = DataLoader(test_set, batch_size = config["batchSize"], num_workers = config["threads"])

### Define the Classifier

In [147]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import  BertModel, BertTokenizer
from torch.utils.data import DataLoader
import torch.optim as optim
import os

class SentimentClassifier(nn.Module):
    def __init__(self, num_classes, device, freeze_bert = True):
        super(SentimentClassifier, self).__init__()
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        self.device = device

        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False

        self.cls_layer = nn.Linear(768, num_classes)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        cont_reps, _ = self.bert_layer(seq, attention_mask = attn_masks)

        #Obtaining the representation of [CLS] head
        cls_rep = cont_reps[:, 0]

        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(cls_rep)

        return logits.to(self.device)

In [103]:
config["device"]

device(type='cuda')

In [105]:
# We are unfreezing the BERT layers so as to be able to fine tune and save a new BERT model that is specific to the Sizeable food reviews dataset.
net = SentimentClassifier(num_classes, config["device"], freeze_bert=False)

In [144]:
net.to(config["device"])
weights = torch.tensor(scores_dist).to(config["device"])

In [148]:
# Setting the Loss function and Optimizer.
loss_func = nn.NLLLoss(weight=weights)
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [149]:
m = nn.LogSoftmax(dim=1)

### Training

In [150]:
def trainFunc(net, loss_func, opti, train_loader, test_loader, config):
    best_acc = 0
    for ep in range(config["epochs"]):
        for it, (seq, attn_masks, labels) in enumerate(train_loader):
            opti.zero_grad()
            #seq, attn_masks, labels = seq.cuda(args.gpu), attn_masks.cuda(args.gpu), labels.cuda(args.gpu)
            seq.to(config["device"])
            attn_masks.to(config["device"])
            labels.to(config["device"])

            logits = net(seq, attn_masks)
            loss = loss_func(m(logits), labels)

            loss.backward()
            opti.step()
            print("Iteration: ", it+1)

            if (it + 1) % config["printEvery"] == 0:
                acc = get_accuracy(m(logits), labels)
                if not os.path.exists(config["outputFolder"]):
                    os.makedirs(config["outputFolder"])

                # Since a single epoch could take well over hours, we regularly save the model even during evaluation of training accuracy.
                torch.save(net.state_dict(), os.path.join(config["outputFolder"], config["outputFileName"]))
                print("Iteration {} of epoch {} complete. Loss : {} Accuracy : {}".format(it+1, ep+1, loss.item(), acc))

        # perform validation at the end of an epoch.
        val_acc, val_loss = evaluate(net, loss_func, val_loader, config)
        print(" Validation Accuracy : {}, Validation Loss : {}".format(val_acc, val_loss))
        if val_acc > best_acc:
            print("Best validation accuracy improved from {} to {}, saving model...".format(best_acc, val_acc))
            best_acc = val_acc
            torch.save(net.state_dict(), os.path.join(config["outputFolder"], config["outputFileName"] + "_valTested_" + str(best_acc)))


In [151]:
trainFunc(net, loss_func, opti, train_loader, val_loader, config)

RuntimeError: ignored